# Консультация 2021
## Подготовил Махнач Фёдор, tg: @fmakhnach

# Структуры

Структуры это т.н. **значимые** типы. Их особенность: память под них выделяется прямо на стеке (для локальных переменных и параметров) **ИЛИ** прямо в памяти объекта (для полей). 

В противоположность, объекты ссылочных типов (class) размещаются в куче, а в стеке (или внутри других объектов) располагаются ссылки на эти объекты.

Вытекающие:
- Не управляются garbage collector'ом;
- Время жизни ограничено областью видимости (scope);
- При передаче в методы экземпляры структур **копируются (!)**;
- Нужно что-то делать с инициализацией по умолчанию, т.к. значение null неприменимо.
- **Не наследуются!** Мы точно должны знать размер экземпляра структуры, что невозможно, если допустить наследование;

## 1. Память выделяется прямо "на месте"

In [1]:
struct Struct
{
    public string Value { get; set; }
}

In [2]:
class MyClass
{
    // Память выделяется на всю структуру прямо внутри объекта MyClass.
    private Struct value;
    
    // Память внутри объекта выделяется только на ссылку!
    private object obj;
    
    public void Method()
    {
        Struct localValue; // Память выделилась на стеке
    }
}

## 2.  Время жизни ограничено scope'ом

In [3]:
void MyMethod()
{

    Struct @struct = new Struct { Value = "Hello" };
    
    object obj = new object();
    
} // В этот момент @struct "умирает", а obj может всё ещё существовать в памяти!

## 3. Передача структур в методы

In [4]:
class Class
{
    public string Value { get; set; }
}

// Копируем ссылку: можем обращаться к полям исходного объекта.
void ChangeValue(Class obj)
{
    obj.Value = "New value!";
}

Class @class = new Class{ Value = "Hello" };
ChangeValue(@class);
@class.Value

New value!

In [5]:
// Копируем весь инстанс: нет доступа к исходному объекту.
void ChangeValue(Struct obj)
{
    obj.Value = "New value!";
}

Struct @struct = new Struct{ Value = "Hello" };
ChangeValue(@struct);
@struct.Value

Hello

Если очень хочется, то можно через ref

In [6]:
// Копируем весь инстанс: нет доступа к исходному объекту.
void ChangeValue(ref Struct obj)
{
    obj.Value = "New value!";
}

Struct @struct = new Struct{ Value = "Hello" };
ChangeValue(ref @struct);
@struct.Value

New value!

## 4. "Конструктор без параметров"

In [1]:
struct Rational
{
    public long Numerator { get; }
    public long Denominator { get; } // = 1;
    
    // public Rational(){}
    
    public Rational(long numerator, long denominator)
    {
        Numerator = numerator;
        Denominator = denominator;
    }
    
    public override string ToString() => $"{Numerator}/{Denominator}";
}

Unhandled Exception: (4,17): error CS0573: '"Rational": в структуре не могут содержаться инициализаторы свойств или полей экземпляров.

In [8]:
object obj;
obj

In [9]:
Rational value;
value.ToString()

0/0

In [10]:
// Вызывать пустой конструктор 10000 раз?
Rational[] values = new Rational[10000];

Отсылаю к [ответу Джона Скита на SO:](https://stackoverflow.com/a/333840/15753334)

> The parameterless constructor isn't created by the compiler. Value types don't have to have constructors as far as the CLR is concerned - although it turns out it can if you write it in IL. When you write "new Guid()" in C# that emits different IL to what you get if you call a normal constructor.

И ещё одна цитата из [другого вопроса](https://stackoverflow.com/questions/203695/does-using-new-on-a-struct-allocate-it-on-the-heap-or-stack), посвящённого "конструкторам без параметров" в структурах

> ... according to C#, all value types have a parameterless constructor. According to the CLI spec, *no* value types have parameterless constructors. (Fetch the constructors of a value type with reflection some time - you won't find a parameterless one.)

**!!!"Позиция партии" в этом вопросе отличается!!!**
<img src="image1.png" style="width: 500px;">

### Резюмируя:
- В структурах нельзя определять конструктор без параметров. Пустой конструктор не генерируется при компиляции: его просто нет. **Но в ваших презентациях сказано иное - учитывайте это на тесте.** 
- При создании экземпляров структур без явного вызова конструктора с параметрами под структуру просто выделяется память и заполняется нулями.
- Как следствие, недопустима inline инициализация полей и свойств.

## 5. Упаковка (boxing)

Иерархия наследования всех структур: STRUCT -> System.ValueType -> System.Object

Значит, можем приводить экземпляры структур к типу Object. Так как структура - значимый тип, а object - ссылочный, требуется особый механизм приведения, который называется **упаковка (boxing)**. Экземпляр структуры "оборачивается" в ссылочный object и помещается в кучу. Это достаточно затратный процесс, поэтому считается плохой практикой.

In [11]:
struct Struct
{
    public string Value { get; set; }
}

public void ChangeValue(Object obj)
{
    // Обратный процесс - распаковка.
    // Значение копируется в локальную переменную.
    Struct s = (Struct)obj;
    s.Value = "New message";
}

Struct s = new Struct { Value = "Message" };
ChangeValue(s); // Упаковка
s.Value

Message

In [12]:
int five = 5;
// Это тоже упаковка!
ValueType valueType = five;

## 6. Наследование

Понимание того, что структуры не наследуются приводит к довольно широким выводам:

- Модификаторы sealed, abstract, virtual, protected запрещены, т.к. не имеют смысла;
- Структура не может быть статической (не имеет смысла + в IL коде `static class` преобразуется в `abstract sealed class`);

In [13]:
sealed struct Sealed {}

static struct Struct
{
    protected int protectedField;

    public abstract void AbstractMethod();
    
    public virtual void VirtualMethod() {}
}

Unhandled Exception: (1,15): error CS0106: Модификатор "sealed" недопустим для этого элемента.
(3,15): error CS0106: Модификатор "static" недопустим для этого элемента.
(5,19): error CS0666: '"Struct.protectedField": новый защищенный член объявлен в структуре.
(7,26): error CS0106: Модификатор "abstract" недопустим для этого элемента.
(9,25): error CS0106: Модификатор "virtual" недопустим для этого элемента.

**И снова неправда в презентациях, можно переопределять методы object (напр. ToString)**

<img src="image2.png" style="width: 500px;">

## 7. Реализация интерфейсов

Структуры могут реализовывать интерфейсы. 

Однако т.к. интерфейсы **всегда являются ссылочными типами**, при создании интерфейсной ссылки на инстанс структуры, этот инстанс всегда упаковывается. Что может довольно сильно ударить по производительности, если это не учитывать.

In [14]:
interface IntValueKeeper
{
    int Value { get; set; }
}

struct Struct : IntValueKeeper
{
    public int Value { get; set; }
}

In [15]:
void ChangeValue(IntValueKeeper obj, int newValue)
{
    obj.Value = newValue;
}

Struct @struct = new Struct { Value = 24 };

// Тут инстанс структуры копируется и упаковывается, что довольно плохо.
IntValueKeeper boxedStruct = @struct; 

// Передали как ссылочный тип
ChangeValue(boxedStruct, 42);

// boxedStruct - ссылочный объект, поэтому значение поменялось!
boxedStruct.Value

42

In [16]:
// Изначальная структура никак не поменялась
@struct.Value

24

In [17]:
// И опять - сравнение ссылок выдаёт false
(IComparable<int>)42 == (IComparable<int>)42

False

## 8. Сравнение структур

У структур не определен оператор сравнения ==, однако можно сравнивать экземпляры структур с помощью .Equals() - причём по значению, а не по ссылке.

In [6]:
struct Point
{
    public double X { get; set; }
    public double Y { get; set; }
}

In [7]:
Point p1 = new Point { X = 42, Y = 24 };
Point p2 = new Point { X = 42, Y = 24 };

In [8]:
p1 == p2

Unhandled Exception: (1,1): error CS0019: Оператор "==" невозможно применить к операнду типа "Point" и "Point".

In [9]:
p1.Equals(p2)

True

## 9. ref, readonly 

**Модификатор ref запрещает экземплярам структуры временно перейти в управляемую кучу (запрещает boxing).**

Это достигается ограничениями на этапе компиляции.

https://docs.microsoft.com/ru-ru/dotnet/csharp/language-reference/builtin-types/struct#ref-struct

In [22]:
public ref struct RefStruct // : IntValueKeeper
{
    public int Value { get; set; }
}

In [23]:
RefStruct[] refStructs = new RefStruct[10];

Unhandled Exception: (1,1): error CS0611: Элементы массива не могут иметь тип "RefStruct".
(1,30): error CS0611: Элементы массива не могут иметь тип "RefStruct".

In [24]:
class MyClass
{
    RefStruct refStructField;
}

Unhandled Exception: (3,5): error CS8345: Поле или автоматически реализуемое свойство не может быть типа "RefStruct", если это не член экземпляра ссылочной структуры.

In [25]:
object obj = (object)new RefStruct();

Unhandled Exception: (1,14): error CS0030: Не удается преобразовать тип "RefStruct" в "object"

In [26]:
List<RefStruct> list;

Unhandled Exception: (1,17): error CS0306: Тип "RefStruct" не может использоваться в качестве аргумента типа

**Модификатор readonly** запрещает экземплярам структуры изменяться после инициализации. Все поля должны быть отмечены readonly, свойства не должны имет setter (могут иметь init, начиная с C# 9.0).

In [27]:
public readonly struct ReadonlyStruct
{
    public readonly int count;
    
    // Ошибка: должно быть readonly
    public double length;
    
    // Ошибка: set недопустим
    public string Name { get; set; }
}

Unhandled Exception: (6,19): error CS8340: Поля экземпляров в структурах только для чтения должны быть доступны только для чтения.
(9,19): error CS8341: Автоматически реализуемые свойства экземпляра в структурах только для чтения должны быть доступны только для чтения.

Причины пользоваться неизменяемыми типами данных:
- Потокобезопасность
- Надёжность
- Инкапсуляция
- Простота

## 10. Nullable

Структуры можно делать Nullable, т.е. добавить поддержку значения null. 

In [28]:
Nullable<int> nullableIntValue = 3;
nullableIntValue

3

In [29]:
// Упрощённый синтаксис того же самого
int? nullableIntNull = null;
nullableIntNull

Для проверки существования значения используется свойство HasValue, для получения значения - Value.

In [30]:
nullableIntValue.HasValue

True

In [31]:
nullableIntValue.Value

3

In [32]:
nullableIntNull.HasValue

False

Если попробуете достать значение которого нет - получите исключение.

In [33]:
nullableIntNull.Value

Unhandled Exception: Nullable object must have a value.

**Nullable<T> это всё ещё структура!**

## 11. Общие рекомендации

- Ввиду особенностей структур, не рекомендуется создавать структуры больших размеров (>16 байт).
- Следует помнить о дорогих операциях упаковки-распаковки, особенно при реализации интерфейсов.
- Обычно структуры используют для определения легковесных типов без сложного состояния и поведения. Хорошие примеры - координаты на плоскости или рациональное число.
- Желательно делать их immutable (неизменяемыми). Для этого есть модификатор readonly.

# Делегаты

**Делегаты** являются реализацией механизма *обратного вызова* в C# и CLR.

Делегат - тип, экземпляры которого способны хранить и вызывать методы с фиксированным набором параметров и типом возвращаемого значения.

 ## 1. Пример

In [1]:
// Пример объявления делегата
public delegate double BinaryOperation(double left, double right);

In [2]:
double CalculateMark(double accumulated, double exam)
{
    if(exam < 4.0)
    {
        return exam;
    }
    return 0.4 * exam + 0.6 * accumulated; 
}

In [3]:
int[] GetResultMarks((double Accumulated, double Exam)[] studentMarks, BinaryOperation formula)
{
    int[] result = new int[studentMarks.Length];
    if (formula is null) 
    {
        return result;
    }
    for(int i = 0; i < studentMarks.Length; i++)
    {
        // Тут на самом деле вызывается formula.Invoke(...)
        double formulaResult = formula(studentMarks[i].Accumulated, studentMarks[i].Exam);
        result[i] = (int)Math.Round(formulaResult, MidpointRounding.AwayFromZero);
    }
    return result;
}

In [4]:
var marks = new (double Accumulated, double Exam)[] { (3.7, 6.5), (9.8, 7.4), (8.4, 9.2) };

// На самом деле это разворачивается в GetResultMarks(marks, new BinaryOperation(CalculateMark))
GetResultMarks(marks, CalculateMark)

index,value
0,5
1,9
2,9


## 2. Внутренности

Любой делегат под капотом представляет из себя специальный класс. Например, для делегата
```
public delegate double BinaryOperation(double left, double right);
```
на этапе компиляции создастся примерно такой класс:

In [ ]:
internal class BinaryOperation : System.MulticastDelegate
{
    public Feedback(Object obj, IntPtr method);
    
    public virtual double Invoke(double left, double right);
    
    public virtual IAsyncResult BeginInvoke(double left, double right, AsyncCallback callback, Object object);
    
    public virtual void EndInvoke(IAsyncResult result);
}

## 3. Ковариация и контравариация

Делегаты поддерживают ковариацию и контравариацию **для ссылочных типов** при привязке методов к делегату.

**Ковариация** позволяет привязать к делегату метод с типом возвращаемого значения, *производного* от типа, возвращаемого делегатом.

**Контравариация** позволяет привязать к делегату метод с типом параметра, *базового* для типа параметра делегата.

Пример ковариации и контравариации в одном флаконе:

In [6]:
using System.IO;

// Обратите внимание на типы
public delegate object MyCallback(FileStream s);

public class MyClass
{
    // Обратите внимание на типы
    public string SomeMethod(Stream s)
    {
        // do something
        
        return string.Empty;
    }
}

In [12]:
public void Method(object obj) => Console.WriteLine(obj.ToString());

Action<string> k = Method;

k("hello")

hello


In [7]:
MyClass obj = new MyClass();

// Успешно компилируется
MyCallback callback = obj.SomeMethod;

**За этим стоит следующая логика:**

Делегат возвращает object => метод возвращает string => string это object => можно засунуть этот метод в делегат. \[*ковариация*\]

При вызове в делегат будет передан FileStream => метод принимает Stream => FileStream это Stream => можно засунуть этот метод в делегат. \[*контравариация*\]

## 4. Из каждого экземпляра делегата можно достать:
- Target - объект, к которому применяется сохранённый метод, null если метод статический;
- Method - объект типа MethodInfo, представляющий информацию о методе.

In [8]:
callback.Target == obj // Это и есть ссылка на исходный объект

True

In [9]:
var methodInfo = callback.Method;
methodInfo.Name

SomeMethod

## 5. Цепочки делегатов

В один делегат можно запихнуть несколько делегатов, они вызовутся в порядке добавления. Если делегат имеет возвращаемое значение, вернётся результат последнего метода.

In [10]:
public delegate int MyCallback();

public static int Oh()
{
    Console.WriteLine("Oh");
    return 1;
}

public int My()
{
    Console.WriteLine("My");
    return 2;
}

public int God()
{
    Console.WriteLine("God");
    return 3;
}

In [11]:
MyCallback callback = Oh;
callback += My; // Под капотом: callback = (MyCallback)Delegate.Combine(callback, My);
callback += God;

int returnValue = callback();

returnValue

Oh
My
God


3

Можно "отменять подписку". Удаляются делегаты с конца.

In [12]:
MyCallback callback;
callback += Oh;
callback += My;
callback += Oh;
callback += My;

callback -= Oh;

callback -= God; // Если метода не было в списке, ничего не произойдёт

callback()

Oh
My
My


2

## 6. Action и Func

В реальности новые типы делегатов объявлять не приходится, т.к. существуют обобщённые делегаты System.Action и System.Func, которые покрывают все потребности.

Их объявления выглядят примерно так:

In [13]:
public delegate void Action<T1, T2, ... T16>(T1 arg1, T2 arg2, ... T16 arg16);

public delegate TResult Func<T1, T2, ... T16, TResult>(T1 arg1, T2 arg2, ... T16 arg16);

Unhandled Exception: (1,37): error CS1001: Требуется идентификатор.
(1,37): error CS8635: Неожиданная последовательность символов "…"
(1,64): error CS1031: Требуется тип.
(1,64): error CS1001: Требуется идентификатор.
(1,64): error CS8635: Неожиданная последовательность символов "…"
(1,68): error CS1003: Синтаксическая ошибка, требуется ","
(3,38): error CS1001: Требуется идентификатор.
(3,38): error CS8635: Неожиданная последовательность символов "…"
(3,74): error CS1031: Требуется тип.
(3,74): error CS1001: Требуется идентификатор.
(3,74): error CS8635: Неожиданная последовательность символов "…"
(3,78): error CS1003: Синтаксическая ошибка, требуется ","

In [14]:
public static string Repeat(string str, int ntimes) => string.Concat(Enumerable.Repeat(str, ntimes));

Func<string, int, string> myFunc = Repeat;

myFunc("abcd", 10)

abcdabcdabcdabcdabcdabcdabcdabcdabcdabcd

In [15]:
public static void SayHello(int ntimes)
{
    for(int i = 0; i < ntimes; i++)
    {
        Console.WriteLine("Hello!");
    }
}

In [16]:
Action<int> action = SayHello;

action(3)

Hello!
Hello!
Hello!


# Лямбда-выражения

Причина введения лямбда выражений - громоздкий синтаксис использования делегатов. Зачастую мы хотим передать небольшую функцию, которая не имеет смысла вне контекста.

In [13]:
// Анонимная функция - старый синтаксис из мезозоя (до C# 2.0)
Func<int, int, bool> dontWriteLikeThis = delegate(int left, int right) { 
    double dist = Math.Abs(right - left);
    return dist > 5;
};

Func<int, int, bool> func1 = (int left, int right) => { 
    double dist = Math.Abs(right - left);
    return dist > 5;
};

// Если компилятор сам может определить типы, их можно не писать
Func<int, int, bool> func2 = (left, right) => Math.Abs(right - left) > 5;

Unhandled Exception: (13,42): error CS1003: Синтаксическая ошибка, требуется ","
(13,45): error CS1002: Требуется ";".

In [18]:
// Для одного параметра можно опустить скобки
Action<string> print = str => Console.WriteLine(str);

print("Hello world")

Hello world


Скобки можно опустить только когда у нас 1 параметр без явного указания типа:

In [19]:
// Можно
Action<string> print1 = str => Console.WriteLine(str);

In [14]:
// Неможно
Action<string> print2 = (string str) => Console.WriteLine(str);

In [21]:
// Неможно
Action<string, string> print3 = str1, str2 => Console.WriteLine(str1 + str2);

Unhandled Exception: (2,44): error CS1003: Синтаксическая ошибка, требуется ","
(2,47): error CS1002: Требуется ";".

Лямбды нельзя присваивать неявно типизированным переменным. Впрочем, обычно вы определяете лямбду прямо при вызове метода, так что это не нужно.

In [22]:
var doesntCompile = (int left, int right) => Math.Abs(right - left) > 5;

Unhandled Exception: (1,5): error CS0815: Не удается присвоить лямбда-выражение неявно типизированной переменной.

### Захват переменных

In [1]:
Action action;

for (int i = 0; i < 5; i++)
{
    action += () => Console.WriteLine(i);
}

action.Invoke()

5
5
5
5
5


# События

- Сущность
- Допустимые операции
- Методы аксессоры
- Общепринятые правила использования
- Пример

## 1. Сущность

Событие (event) - обёртка над делегатом, предоставляющая более безопасное использование.

Объявляется как обычный член класса, например:

In [23]:
public class EventHolder
{
    public event EventHandler<EventArgs> MyUselessEvent;
}

## 2. Допустимые операции

Допустимые операции **извне класса**:
- Подписать метод на событие (+=);
- Отписать метод от события (-=);

Всё. Нельзя **вызвать событие** или **присвоить ему новое значение** - это прерогатива самого класса. Это и отличает события от делегатов. 

In [24]:
public class EventHolder
{
    public event EventHandler<EventArgs> MyUselessEvent;
    
    public void TriggerEvent(EventArgs e)
    {
        MyUselessEvent?.Invoke(this, e);
    }
}

In [25]:
EventHolder ev = new EventHolder();

ev.MyUselessEvent += (sender, e) => Console.WriteLine("Hello");
ev.MyUselessEvent += (sender, e) => Console.WriteLine("World");

// Не получится изменить значение
// ev.MyUselessEvent = (sender, e) => Console.WriteLine("Goodbye");

// Не получится вызвать напрямую
// ev.MyUselessEvent?.Invoke()

ev.TriggerEvent(EventArgs.Empty)

Hello
World


## 3. Методы аксессоры

Свойство - эдакое свойство, но для делегатов. 

В реальности событие разворачивается в приватное поле делегата и 2 метода **add** и **remove**, соответствующие операциям "+=" и "-=" соответственно.

- Их можно переопределить, но как правило это не требуется.
- Вы не можете переопределить один - нужны оба!
- **Переопределение add и remove отнимает возможность вызывать событие "напрямую"!**

In [16]:
public class EventHolder
{
    private EventHandler<EventArgs> myEvent; 
    
    public event EventHandler<EventArgs> MyUselessEvent
    {
        add 
        { 
            myEvent += value;
            Console.WriteLine("Adding new method"); 
        }
        remove 
        { 
            myEvent -= value;
            Console.WriteLine("Removing method"); 
        }
    }
    
    public void TriggerEvent(EventArgs e)
    {
        // Так нельзя, раз переопределены аксессоры!
        // MyUselessEvent?.Invoke(this, e);
        
        myEvent?.Invoke(this, e);
    }
}

In [18]:
EventHolder ev = new EventHolder();

EventHandler<EventArgs> a = (sender, e) => Console.WriteLine("Do nothing");

ev.MyUselessEvent += a;
ev.MyUselessEvent -= a;

ev.TriggerEvent(EventArgs.Empty);

Adding new method
Removing method


## 4. Общепринятые правила использования

**Invocation list**, который в делегатах применяется не очень часто, очень полезен в событиях: предполагается, что на событие может подписаться много методов. Поэтому мы не хотим создавать события с возвращаемым значением, отличным от void - в случае вызова нескольких методов мы получим результат лишь последнего.

Можно использовать одну из вариаций Action, но классическим считается использование делегата EventHandler<TEventArgs>, сигнатура которого выглядит так:

In [28]:
public delegate void EventHandler<TEventArgs>(object sender, TEventArgs e);

Ограничений на ```TEventArgs``` не накладывается, хотя могли бы. Предполагается, что будет использоваться наследник EventArgs, примерно так:

In [29]:
public class MessageEventArgs : EventArgs
{
    public string Message { get; set; }
}

In [30]:
public class EventHolder
{
    public event EventHandler<MessageEventArgs> MyUselessEvent;
    
    public void TriggerEvent(MessageEventArgs e)
    {
        MyUselessEvent?.Invoke(this, e);
    }
}

In [31]:
EventHolder ev = new EventHolder();

ev.MyUselessEvent += (sender, e) => Console.WriteLine("1. " + e.Message); 
ev.MyUselessEvent += (sender, e) => Console.WriteLine("2. " + e.Message);

ev.TriggerEvent(new MessageEventArgs { Message = "You do not talk about Fight Club" });

1. You do not talk about Fight Club
2. You do not talk about Fight Club


# Интерфейсы

Интерфейс - контракт, по которому класс, его реализующий, предоставляет какие-то методы.

Написание кода с опорой на интерфейсы, а не на конкретные типы позволяет:
- **Переиспользовать код, абстрагируясь от реализации.** Один раз написанный алгоритм сортировки элементов, опирающийся только на интерфейс IComparable, одинаково работает как со встроенными типами, так и с вашими.
- **Подменять реализацию, в том числе во время исполнения.**
- **Сделать код более безопасным.** Объект, передаваемый по интерфейсной ссылке предоставляет только ограниченную информацию о своих возможностях.
- **Не опасаться за последствия (по сравнению с наследованием).** Так как мы не тянем за собой реализацию, не возникает проблем, как с множественным наследованием.

## 1. Правила определения интерфейсов

В интерфейсе определяются сигнатуры *экземплярных функциональных* членов класса, кроме конструкторов. 

Т.е. недопустимы
- Поля
- Конструкторы
- Что-либо статическое

Всё остальное - можно:
- Методы
- Свойства
- События
- Индексаторы

Модификатор доступа не указывается - он априори public.

In [31]:
public interface ISomethingMessy
{
    // Метод
    void Execute();
    
    // Свойство
    string Message { get; }
    
    // Индексатор
    object this[int index] { get; set; }
    
    // Событие
    event Action MyEvent;
}

Unhandled Exception: (11,31): error CS0073: Методы доступа add и remove должны иметь тело.
(11,39): error CS0073: Методы доступа add и remove должны иметь тело.

Пример из стандартной библиотеки - System.IDisposable
```
public interface IDisposable
{
    void Dispose();
}
```

## 2. Реализация интерфейсов. Наследование

In [3]:
using System.IO;

class Base : IDisposable
{
    private FileStream fileStream;
    
    // ...
    
    // public void Dispose() { fileStream.Dispose(); }
}

Unhandled Exception: (3,14): error CS0535: '"Base" не реализует член интерфейса "IDisposable.Dispose()".

In [30]:
using System.IO;

class Base : IDisposable
{
    private FileStream fileStream;
    
    // ...
    
    public void Dispose() { fileStream.Dispose(); }
}

class Derived : Base
{
    // ...
}

// Все наследники класса автоматически реализуют интерфейсы родителя.
Derived derived = new Derived();
derived is IDisposable

True

## 3. Также доступны методы класса object

In [5]:
IComparable<int> value = 3;
value.ToString()

3

In [6]:
value.GetType()

System.Int32

## 4. ~~Реализация~~ Наследование интерфейсов интерфейсами

Можно расширить интерфейс, отнаследовав от него другой интерфейс. Типы, реализующие интерфейс-ребёнок будут обязаны реализовать функционал обоих интерфейсов.

**Однако это оправдано тогда и только тогда, когда жёсткая связь допустима.**

Иначе лучше использовать несколько маленьких интерфейсов согласно **Interface Segregation Principle**.

In [7]:
public interface IVehicle
{
    void MoveTo(float x, float y, float z);
}

public interface IWheeledVehicle : IVehicle
{
    int NumOfWheels { get; }
}

public class Car : IWheeledVehicle { }


Unhandled Exception: (11,20): error CS0535: '"Car" не реализует член интерфейса "IWheeledVehicle.NumOfWheels".
(11,20): error CS0535: '"Car" не реализует член интерфейса "IVehicle.MoveTo(float, float, float)".

Пример наследования интерфейсов из стандартной библиотеки - IEnumerable

```
public interface IEnumerable<out T> : IEnumerable
{
    IEnumerator<T> GetEnumerator();
}

public interface IEnumerable
{
    IEnumerator GetEnumerator();
}
```

## 5. Явная (explicit) и неявная (implicit) реализации интерфейса

Однако можно реализовать интерфейс, не предоставив публичную реализацию методов.

Этого можно добиться, реализовав интерфейс **явно** (explicit). Такая реализация будет доступна **только по соответствующей интерфейсной ссылке**.

In [8]:
public class MyClass : IDisposable
{
    // Неявная реализация интерфейса
    // public void Dispose() { Console.WriteLine("Implicit"); }

    // Явная реализация интерфейса
    void IDisposable.Dispose() { Console.WriteLine("Explicit"); }
}

In [9]:
MyClass myClass = new MyClass();
myClass.Dispose();

Unhandled Exception: (2,9): error CS1061: "MyClass" не содержит определения "Dispose", и не удалось найти доступный метод расширения "Dispose", принимающий тип "MyClass" в качестве первого аргумента (возможно, пропущена директива using или ссылка на сборку).

In [10]:
IDisposable disposable = new MyClass();
disposable.Dispose();

Explicit


**В чём смысл?**

Можно реализовать несколько интерфейсов, содержащих несколько одинаковых по сигнатуре методов. Если они представляют одинаковый смысл то проблем не возникает - а если они в сущности разные?

С помощью явных реализаций интерфейса можно определить **разное поведение** экземпляра в зависимости от того, по какой ссылке мы вызываем интерфейсный метод.

P.S. Пример супер надуманный

In [20]:
// "Исполнитель"
public interface IExecutor
{
    void Execute();
}

In [21]:
// "Палач"
public interface IExecutioner
{
    void Execute();
}

In [22]:
public class Officer : IExecutor, IExecutioner
{
    public void Execute() { /* some boring actions */ Console.WriteLine("Job executed."); }
    
    void IExecutor.Execute() { /* some murderous actions */ Console.WriteLine("dsasd executed."); }
    
    void IExecutioner.Execute() { /* some murderous actions */ Console.WriteLine("Intruder executed."); }
}

In [23]:
Officer officer = new Officer();
officer.Execute();

Job executed.


In [24]:
IExecutor executor = officer;
executor.Execute();

dsasd executed.


In [25]:
IExecutioner executioner = officer;
executioner.Execute();

Intruder executed.


## 6. Обобщённые интерфейсы

Интерфейсы могут быть обобщёнными, таким образом получив все преимущества обобщений.

Из приятного: можно реализовать один и тот же интерфейс с различными параметрами типа, т.к. *как вы знаете*, обобщённые типы с разными параметрами конструируются в разные типы.

In [17]:
public class Number : IComparable<int>, IComparable<double>, IComparable<string>
{
    private int Value { get; }
    
    public Number(int number)
    {
        Value = number;
    }

    public int CompareTo(int other) 
    {
        Console.WriteLine("Hello from int");
        return Value.CompareTo(other);
    }
    
    
    public int CompareTo(double other)
    {
        Console.WriteLine("Hello from double");
        return ((double)Value).CompareTo(other);
    }
    
    public int CompareTo(string other)
    {
        Console.WriteLine("Hello from string");
        return ((double)Value).CompareTo(double.Parse(other));
    }
}

In [18]:
Number number = new Number(42);

In [19]:
number.CompareTo(13)

Hello from int


1

In [20]:
number.CompareTo(42.5)

Hello from double


-1

In [21]:
number.CompareTo("42")

Hello from string


0

Можно использовать интерфейсы в ограничениях на аргумент-тип. Если использовать несколько, то аргумент-тип должен реализовать все.

In [26]:
public void SayHello<T>(T value) where T : IComparable<int>, IDisposable
{
    Console.WriteLine("Hello!");
}

In [27]:
public class MyClass : IComparable<int>, IDisposable
{
    public int CompareTo(int other) => throw new NotImplementedException();
    
    public void Dispose() => throw new NotImplementedException();
}

In [28]:
MyClass obj = new MyClass();
SayHello(obj)

Hello!


## 7. Реализация метода интерфейса по умолчанию

Начиная с C# 8.0 можно определять реализацию методов интерфейса по умолчанию.

Такая реализация доступна только по интерфейсной ссылке

In [25]:
public interface ISummator
{
    int Sum(IEnumerable<int> values) 
    {
        int result = 0;
        foreach(var value in values)
        {
            result += value;
        }
        return result;
    }
}

In [26]:
public class MySummator : ISummator
{
}

In [27]:
MySummator mySummator = new MySummator();

mySummator.Sum(new int[]{1, 2, 3, 4, 5})

Unhandled Exception: (3,12): error CS1061: "MySummator" не содержит определения "Sum", и не удалось найти доступный метод расширения "Sum", принимающий тип "MySummator" в качестве первого аргумента (возможно, пропущена директива using или ссылка на сборку).

In [28]:
ISummator summator = new MySummator();

summator.Sum(new int[] { 1, 2, 3, 4, 5 })

15

## 8. Абстрактный класс или интерфейс?

**Абстрактный класс:**
- Является классом, а значит наследуясь от него нельзя наследоваться от других классов;
- Может определять часть состояния и поведения;
- Наследование - очень сильная связь;

Абстрактный определяет каркас для нескольких различных реализаций сущности.

**Интерфейс:**
- Класс может реализовывать сколько угодно интерфейсов;
- Определяет (в общем случае) только *что* должен делать класс, но не *как* (в общем случае);
- Реализация интерфейс - слабая связь;

Интерфейс определяет набор свойств, которыми должна обладать сущность, её некоторый обособленный функционал.

# Обобщения

Одна из сильных сторон C# - мощные типобезопасные обобщения, позволяющие использовать в качестве аргументов типа как ссылочные, так и значимые типы.

- Мотивация
- Внутреннее устройство
- Наследование
- Ковариантность и контравариантность типов
- Ограничения

# Мотивация

Очень часто хочется противоречащие вещи:

1. Один раз написать некоторый алгоритм/структуру данных так, чтобы она работала с разными типами данных.
2. Сохранить безопасность типов, чтобы избавиться от гемороя с обработкой ошибок типов.

Например, структура данных "Динамический массив" (список) по идее не зависит от типа содержимого. Мы не хотим переписывать этот код для каждого типа, с которым мы используем список.

Можно конечно сделать список object'ов (привет дженерикам из джавы), но так мы не сохраняем безопасность типов.

In [ ]:
public class MyList
{
    private const int DefaultCapacity = 4;

    private int[] data;
    private int capacity;
    
    public int Count { get; private set; }
    
    public int this[int index]
    {
        get
        {
            if(index < 0 || index >= Count) throw new IndexOutOfRangeException($"Got index {index}, but size is {Count}.");
            return data[index];
        }
        set
        {
            if(index < 0 || index >= Count) throw new IndexOutOfRangeException($"Got index {index}, but size is {Count}.");
            data[index] = value;
        }
    }
    
    public MyList() : this(DefaultCapacity) { }
    
    public MyList(int capacity)
    {
        if(capacity <= 0) throw new ArgumentException("Capacity must be positive");
        
        this.capacity = capacity;
        data = new int[capacity];
        Count = 0;
    }
    
    public void Add(T item)
    {
        if (Count == capacity) {
            Array.Resize(ref data, capacity *= 2);
        }
        data[Count] = item;
        ++Count;
    }
}

А вот и обобщённый вариант:

In [ ]:
public class MyList<T>
{
    private const int DefaultCapacity = 4;

    private T[] data;
    private int capacity;
    
    public int Count { get; private set; }
    
    public T this[int index]
    {
        get
        {
            if(index < 0 || index >= Count) throw new IndexOutOfRangeException($"Got index {index}, but size is {Count}.");
            return data[index];
        }
        set
        {
            if(index < 0 || index >= Count) throw new IndexOutOfRangeException($"Got index {index}, but size is {Count}.");
            data[index] = value;
        }
    }
    
    public MyList() : this(DefaultCapacity) { }
    
    public MyList(int capacity)
    {
        if(capacity <= 0) throw new ArgumentException("Capacity must be positive");
        
        this.capacity = capacity;
        data = new T[capacity];
        Count = 0;
    }
    
    public void Add(T item)
    {
        if (Count == capacity) {
            Array.Resize(ref data, capacity *= 2);
        }
        data[Count] = item;
        ++Count;
    }
}

`T` называется **параметром типа**, подставленное в него значение (напр. `int` в `List<int>`) - **аргументом типа**.

Также вводятся понятия **открытого** и **закрытого** типов.

**Закрытый тип** - обобщённый тип, в котором определены **все** параметры типа.

**Открытый тип** - обобщённый тип, в котором определены **не все** параметры типа.

In [20]:
// Закрытый тип
typeof(Dictionary<long, int>)

System.Collections.Generic.Dictionary<System.Int64,System.Int32>

In [21]:
// Открытый тип
typeof(Dictionary<,>)

System.Collections.Generic.Dictionary<TKey,TValue>

Обобщёнными могут быть: 

- Типы
 - Классы
 - Структуры
 - Интерфейсы
 - Делегаты
- Методы

Короче всё, что можно себе представить.

## Внутреннее устройство

Для каждого нового аргумента типа CLR генерирует **новый отдельный класс**, подставляя в него те аргументы, которые вы передали.

In [5]:
typeof(List<int>).ToString()

System.Collections.Generic.List`1[System.Int32]

In [10]:
typeof(List<double>).ToString()

System.Collections.Generic.List`1[System.Double]

In [4]:
typeof(Dictionary<long, string>).ToString()

System.Collections.Generic.Dictionary`2[System.Int64,System.Int32]

Благодаря этому достигается одно из преимуществ обобщений - **эффективная работа со значимыми типами (структурами)**. Для структур генерируется немного другой код, что позволяет избежать операций упаковки и распаковки.

Ещё одно важное вытекающее: **у каждого конкретного обобщённого типа своё статическое состояние**. 

In [12]:
class MyGeneric<T>
{
    public static int Value { get; set; }
}

In [13]:
MyGeneric<int>.Value = 4;
MyGeneric<double>.Value = 8;

In [14]:
MyGeneric<int>.Value

4

In [15]:
MyGeneric<double>.Value

8

## Наследование

Можно использовать параметры типа при наследовании и реализации интерфейса. 

Можно и не использовать.

In [24]:
public class MyGeneric<T1, T2> : List<T1>
{
    // ...
}

In [28]:
public class MyGeneric<T1, T2> : IComparable<int>
{
    public int CompareTo(int other) => 1;
}

## Ковариантность и контравариантность

Проблема:

string это object. Тогда коллекция string это коллекция object'ов - или нет?

In [40]:
List<string> myStrings = new List<string>() { "Hello", "World!" };
List<object> myObjects = myStrings;

Unhandled Exception: (2,26): error CS0029: Не удается неявно преобразовать тип "System.Collections.Generic.List<string>" в "System.Collections.Generic.List<object>".

А вот так оно почему-то работает. Магия?

In [42]:
IEnumerable<string> myStrings = new List<string>() { "Hello", "World!" };
IEnumerable<object> myObjects = myStrings;

foreach(var obj in myObjects)
{
    Console.WriteLine(obj);
}

Hello
World!


Было string -> object.

А object -> string?

In [45]:
public class ObjectConsumer : IComparable<object>
{
    public int CompareTo(object other) => 1;
}

IComparable<object> objComparable = new ObjectConsumer();
IComparable<string> strComparable = objComparable;

strComparable.CompareTo("Hello, world!")

1

Немного теории

> Параметры обобщённого типа могут быть:
> - **Инвариантными.** Параметр-тип не может изменяться. По умолчанию так.
> - **Ковариантными.** Аргумент-тип может быть преобразован от класса к одному из его базовых классов. В языке С# ковариантный тип обозначается ключевым словом out. Ковариантный параметр обобщенного типа может появляться только в выходной позиции, например, в качестве возвращаемого значения метода.
> - **Контравариантными.** Параметр-тип может быть преобразован от класса к классу, производному от него. В языке C# контравариантный тип обозначается ключевым словом in. Контравариантный параметр-тип может появляться только во входной позиции, например, в качестве аргументов метода.

In [33]:
// Делаем тип ковариативным
public interface IUseless<out T>
{
    T DoNothing(object obj);
}

In [31]:
// Тут сделать ковариативным нельзя, т.к. принимаем на вход.
public interface IUseless<out T>
{
    T DoNothing(T obj);
}

Unhandled Exception: (3,17): error CS1961: Недопустимое отклонение: Параметр типа "T" должен быть контравариантно, допустимым на "IUseless<T>.DoNothing(T)". "T" является ковариантный.

In [34]:
// Делаем тип контравариативным
public interface IUseless<in T>
{
    object DoNothing(T obj);
}

In [30]:
// Тут сделать контравариативным нельзя, т.к. возвращаем из метода.
public interface IUseless<in T>
{
    T DoNothing(T obj);
}

Unhandled Exception: (3,5): error CS1961: Недопустимое отклонение: Параметр типа "T" должен быть ковариантно, допустимым на "IUseless<T>.DoNothing(T)". "T" является контравариантный.

**Ко(нтра)вариация не работает для значимых типов!**

In [51]:
IEnumerable<string> myStrings = new List<string>() { "Hello", "World!" };
IEnumerable<object> myObjects = myStrings;

In [54]:
IEnumerable<int> myInts = new List<int>() { 1, 2, 3 };
// немогу...
IEnumerable<object> myObjects = myInts;

Unhandled Exception: (3,33): error CS0266: Не удается неявно преобразовать тип "System.Collections.Generic.IEnumerable<int>" в "System.Collections.Generic.IEnumerable<object>". Существует явное преобразование (возможно, пропущено приведение типов).

**Ковариация и контравариация расширяют возможности обобщений, но могут быть трудны в понимании.**

## Ограничения аргументов типов (constraints)

Дженерики - довольно жёсткая конструкция, так как одна из первостепенных задач - сохранить безопасность типов.

По умолчанию для объектов неопределённого типа доступны только методы класса object.

In [29]:
public T Min<T>(T left, T right)
{
    if(left.CompareTo(right) < 0)
        return left;
    return right;
}

Unhandled Exception: (3,13): error CS1061: "T" не содержит определения "Compare", и не удалось найти доступный метод расширения "Compare", принимающий тип "T" в качестве первого аргумента (возможно, пропущена директива using или ссылка на сборку).

Можно наложить на параметр-тип некоторые обязательства, а взамен - получить возможность пользоваться какими-то функциями. Например, можете обязать аргумент-тип реализовывать некоторый интерфейс. 

In [58]:
// Любой тип T должен быть IComparable<T>
public T Min<T>(T left, T right) where T : IComparable<T>
{
    if(left.CompareTo(right) < 0)
        return left;
    return right;
}

In [60]:
Min<int>(1, 5)

1

In [62]:
// Вывод типов компилятором - можно не указывать тип явно.
// Фича работает только с методами (по понятным причинам).
Min("bruh", "meh")

bruh

Можно довольно интересно использовать ограничения, связывая несколько параметров типов.

In [31]:
public List<TBase> ConvertIList<T, TBase>(IList<T> list) where T : TBase
{
    List<TBase> baseList = new List<TBase>(list.Count);
    for (Int32 index = 0; index < list.Count; index++) {
        baseList.Add(list[index]);
    }
    return baseList;
}

In [68]:
IList<String> ls = new List<String>() { "A String" };

IList<Object> lo = ConvertIList<String, Object>(ls);

IList<IComparable> lc = ConvertIList<String, IComparable>(ls);

IList<IComparable<String>> lcs = ConvertIList<String, IComparable<String>>(ls);

IList<String> ls2 = ConvertIList<String, String>(ls);

Можно применить следующие ограничения на параметр тип:
- Является классом: `where T : class`
- Является значимым типом: `where T : struct`
- Является типом (или производным от него): `where T : MyClass`
- Реализует интерфейс: `where T : IInterface`
- Имеет пустой конструктор: `where T : new()`

Порядок ниже

<img src="image3.png" style="width: 500px;">

Ограничения на енамы нет, если это нужно (зачем??), можно использовать статический конструктор класса, например.

In [ ]:
class GenericTypeThatRequiresEnum<TEnum>
{
    static GenericTypeThatRequiresEnum()
    {
        if(!typeof(TEnum).IsEnum)
        {
            throw new ArgumentException("TEnum must be an enumerated type!");
        }
    }
}

Интересный факт: ограничение where T : struct пропускает енамы

In [71]:
enum MyEnum 
{
    First, Second
}

public void MyGenericMethod<T>(T obj) where T : struct
{
    Console.WriteLine(typeof(T));
    Console.WriteLine(obj);
}

MyGenericMethod(MyEnum.First);

Submission#72+MyEnum
First


## Что нужно запомнить

- Для каждого конкретного набора аргументов типов генерируется отдельный класс/структура/интерфейс/делегат/метод.
- Сильная сторона - эффективная работа со значимыми (struct) аргументами типа.
- Ковариантность (out) - когда "можем засунуть  *производный* от заданного параметра. Допустим только тогда, когда тип используется лишь в возвращаемых значениях методов. Логика: "если мы возвращали Base, "
- Контравариантность (in) - когда можем использовать тип *базовый* для заданного параметра. Допустимо только тогда, когда тип используется лишь в параметрах методов. Логика: "если мы ожидаем параметр Base, от того, что нам начнут присылать Derived, нарушений не возникнет".
- С помощью ограничений на типы можно определить, какой набор свойств должен иметь аргумент тип. Благодаря этому можно безопасно использовать эти свойства в коде обобщенного класса/метода/чего-либо.

# Перечислимые типы (enums)

## 1. Базовые возможности

In [33]:
enum Color
{
    White,  // 0
    Red,    // 1
    Green,  // 2
    Blue,   // 3
    Orange, // 4
}

In [34]:
Color white = Color.White;  
Console.WriteLine(white);   // White

Color red = (Color)1;       // Так можно приводить к типу перечисления 
Console.WriteLine(red);     // Red

Color unknown = (Color)42;  // Нет ошибки!
Console.WriteLine(unknown); // 42
unknown is Color

White
Red
42


True

In [3]:
Color green = Enum.Parse<Color>("Green");
green.ToString()

Green

In [4]:
Enum.TryParse<Color>("Blue", out Color blue);
blue.ToString()

Blue

In [5]:
// Посмотрим, какими типами можно задавать перечисления
enum Dummy : object {}

Unhandled Exception: (2,14): error CS1008: Требуется тип byte, sbyte, short, ushort, int, uint, long или ulong.

In [36]:
Color[] colors = new Color[] { Color.Red, Color.Blue };

## 2. Приведение перечислимых типов

In [6]:
enum Fruit
{
    Melon,      // 0
    Tomato,     // 1
    Apple,      // 2
    Blueberry,  // 3
    Orange,     // 4
}

In [7]:
Fruit orange = Color.Orange; // Безопасность типов -> ошибка

Unhandled Exception: (1,16): error CS0266: Не удается неявно преобразовать тип "Color" в "Fruit". Существует явное преобразование (возможно, пропущено приведение типов).

In [8]:
Fruit tomato = (Fruit)Color.Red; // А вот так уже можно
Console.WriteLine(tomato);

Tomato


In [9]:
Color unknownColor = (Color)42;
Fruit unknownFruit = (Fruit)unknownColor;
Console.WriteLine(unknownFruit);

42


In [10]:
// Любой enum имеет следующую цепочку наследования: MyEnum <- System.Enum <- System.ValueType <- System.Object

Enum enumEnum = Color.Blue;
ValueType enumValueType = Color.Blue; // BOXING
object enumObj = Color.Blue;          // BOXING

Console.WriteLine($"{enumEnum}, {enumValueType}, {enumObj}");

Blue, Blue, Blue


## 3. Использование одного целочисленного значения для нескольких enum значений

In [37]:
public enum Subject
{
    Programming = 0,
    DiscreteMath = 1,
    Algebra = 2,
    Calculus = 3,
    Economics = 4,

    MostDifficultSubject = Algebra,
    MostUsefulSubject = Programming,
    MostHatefulSubject = Programming
}

In [38]:
Console.WriteLine(Subject.Programming);
Console.WriteLine(Subject.MostUsefulSubject);
Console.WriteLine((Subject)0);

Console.WriteLine(Subject.Programming == Subject.MostUsefulSubject)

Programming
Programming
Programming
True


In [39]:
Console.WriteLine(Subject.Algebra);
Console.WriteLine(Subject.MostDifficultSubject);
Console.WriteLine((Subject)2);

Console.WriteLine(Subject.Algebra == Subject.MostDifficultSubject)

MostDifficultSubject
MostDifficultSubject
MostDifficultSubject
True


## 4. Рефлексия перечислимых типов

Статический метод Enum.GetUnderlyingType возвращает целочисленный тип для енама

In [14]:
Enum.GetUnderlyingType(typeof(Subject))

System.Int32

В типе System.Type также есть метод GetEnumUnderlyingType

In [16]:
typeof(Subject).GetEnumUnderlyingType()

System.Int32

Который работает только с объектами-типами енамов

In [17]:
typeof(short).GetEnumUnderlyingType()

Unhandled Exception: Type provided must be an Enum. (Parameter 'enumType')

Можно получить все значения енама c помощью Enum.GetValues(Type)

In [18]:
var enumValues = Enum.GetValues(typeof(Subject)); // Аналог: typeof(Subject).GetEnumValues();
foreach(var value in enumValues){
    Console.WriteLine(value);
}

Programming
Programming
DiscreteMath
Algebra
Algebra
Calculus
Economics


In [19]:
Enum.GetNames(typeof(Subject)) // Аналог: typeof(Subject).GetEnumNames()

index,value
0,MostUsefulSubject
1,Programming
2,DiscreteMath
3,Algebra
4,MostDifficultSubject
5,Calculus
6,Economics


Проверка, есть ли в енаме соответствующее значение.

In [20]:
Enum.IsDefined(typeof(Subject), 3)

True

In [21]:
Enum.IsDefined(typeof(Subject), 42)

False

## 5. Битовые флаги

In [44]:

enum FilePermission : byte
{
    None    = 0b00000000,

    Read    = 0b00000001,
    Write   = 0b00000010,
    Execute = 0b00000100,
    Rename  = 0b00001000,
    Move    = 0b00010000,
    Delete  = 0b00100000,

    User        = Read | Execute,
    ReadWrite   = Read | Write,
    Admin       = Read | Write | Execute | Rename | Move | Delete
}

[Про FlagsAttribute](https://docs.microsoft.com/ru-ru/dotnet/api/system.flagsattribute?view=net-5.0)

In [45]:
FilePermission permission = FilePermission.User;
permission.HasFlag(FilePermission.Read)

True

Пример использования:
```
void RenameFile(File file, User user)
{
    if (!user.Permission.HasFlag(FilePermission.Rename)) {
        throw new SomeException("you can't.")
    }
    ...
}
```

In [43]:
for (int i = 0; i <= 16; ++i) {
    FilePermission fp = (FilePermission)i;
    Console.WriteLine(fp.ToString("G"));
}

None
Read
Write
ReadWrite
Execute
User
Write, Execute
Write, User
Rename
Read, Rename
Write, Rename
ReadWrite, Rename
Execute, Rename
User, Rename
Write, Execute, Rename
Write, User, Rename
Move


Пример из стандартной библиотеки: System.AttributeTargets

In [25]:
[Flags, Serializable]
public enum AttributeTargets {
    Assembly = 0x0001,
    Module = 0x0002,
    Class = 0x0004,
    Struct = 0x0008,
    Enum = 0x0010,
    Constructor = 0x0020,
    Method = 0x0040,
    Property = 0x0080,
    Field = 0x0100,
    Event = 0x0200,
    Interface = 0x0400,
    Parameter = 0x0800,
    Delegate = 0x1000,
    ReturnValue = 0x2000,
    GenericParameter = 0x4000,
    All = Assembly | Module | Class | Struct | Enum |
    Constructor | Method | Property | Field | Event |
    Interface | Parameter | Delegate | ReturnValue |
    GenericParameter
}

## 6. Методы расширения для enum

Перечислениям можно "добавлять функциональность" с помощью методов расширения

In [26]:
//public static class EnumExtentions
//{
    public static int GetMark(this Subject subject)
    {
        return subject switch 
        {
            Subject.Programming => 8,
            Subject.DiscreteMath => 10,
            Subject.Algebra => 5,
            Subject.Calculus => 7,
            Subject.Economics => 6,
            _ => 0,
        };
    }
//}

In [27]:
Subject prog = Subject.Programming;
prog.GetMark()

8